In [1]:
%pip install beautifulsoup4
%pip install lxml
%pip install selenium

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
import pandas as pd
import time

In [3]:
driver = webdriver.Chrome()
driver.get('https://www.masters.com/en_US/tournament/past_winners.html')
soup = BeautifulSoup(driver.page_source, 'html.parser')
driver.quit()

In [4]:
allscores = soup.find('tbody')
headers = soup.find('thead')

In [5]:
print(headers)

<thead><tr><th class="first-col">Pos</th><th class="first-col">Player</th><th class="cell active">R1</th><th class="cell active">R2</th><th class="cell active">R3</th><th class="cell active">R4</th><th class="cell active">Total Score</th><th class="cell active">Total Par</th></tr></thead>


In [6]:
scores_list = allscores.find_all('tr')
headers_list = headers.find_all('th')

In [7]:
df_headers = [col.text for col in headers_list]

In [8]:
def pull_player_scores(allPlayerScores,heads):
    year_scores = pd.DataFrame(columns = heads)
    for player in allPlayerScores:
        scores = player.find_all('td')
        item_list = [[item.text for item in scores]]
        player_scores = pd.DataFrame(item_list, columns = heads)
        year_scores = pd.concat([year_scores,player_scores])
        
    return(year_scores)

In [9]:
#scores_2020 = pull_player_scores(scores_list, df_headers)

In [9]:
#Lets make a seperate function for extracting the infor we need (allPlayerScores,heads)
def create_lists(page_data):
    allscores = page_data.find('tbody')
    headers = page_data.find('thead')
    
    scores_list = allscores.find_all('tr')
    headers_list = headers.find_all('th')
    
    df_headers = [col.text for col in headers_list]
    
    return(scores_list,df_headers)

In [39]:
#List of Years I want
years_int = list(range(1993,2021))
years = [str(year) for year in years_int]

In [38]:
def pull_one_year(input_year,driver):
    
    #driver = webdriver.Chrome()
    #driver.get('https://www.masters.com/en_US/tournament/past_winners.html')
    
    #close_privacy = driver.find_element_by_xpath("//div[@class= 'privacy-close']//i[@class='icon-close']")
    #close_privacy.click()
    #We only want to loop open the window once when we loop through the yearss so this isn't necessary
    
    
    down_button = driver.find_element_by_xpath("//div[@class = 'title navigation_down_arrow']")
    down_button.click()
    
    time.sleep(0.5)
    
    year_button = driver.find_element_by_xpath(f"//div[@class='year-list list1']//li[text() = {input_year}]")
    year_button.click()
    
    time.sleep(1)
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    #driver.quit()
   
    pulled_scores,pulled_headers = create_lists(soup)
    
    scores_df = pull_player_scores(pulled_scores,pulled_headers)
    scores_df['Year'] = input_year
    
    return(scores_df,driver)

In [19]:
#pull_one_year('2019')

<ipython-input-18-8dc95843d784>:6: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  close_privacy = driver.find_element_by_xpath("//div[@class= 'privacy-close']//i[@class='icon-close']")
<ipython-input-18-8dc95843d784>:10: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  down_button = driver.find_element_by_xpath("//div[@class = 'title navigation_down_arrow']")
<ipython-input-18-8dc95843d784>:13: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  year_button = driver.find_element_by_xpath(f"//div[@class='year-list list1']//li[text() = {input_year}]")


,Pos,Player,R1,R2,R3,R4,Total Score,Total Par,Year
0,1,D Johnson,65,70,65,68,268,-20,2019
0,T2,C Smith,67,68,69,69,273,-15,2019
0,T2,S Im,66,70,68,69,273,-15,2019
0,4,J Thomas,66,69,71,70,276,-12,2019
0,T5,R McIlroy,75,66,67,69,277,-11,2019
...,...,...,...,...,...,...,...,...,...
0,MC,J Olazabal,78,80,,,158,+14,2019
0,MC,A Landry,78,82,,,160,+16,2019
0,MC,A Gallegos,79,81,,,160,+16,2019
0,WD,V Singh,75,,,,,,2019


In [12]:
df_headers.append('Year')

In [37]:
def masters_all_years(list_years,headers):
    masters_results = pd.DataFrame(columns = headers)
    
    driver = webdriver.Chrome()
    driver.get('https://www.masters.com/en_US/tournament/past_winners.html')
    
    time.sleep(0.5)
    
    close_privacy = driver.find_element_by_xpath("//div[@class= 'privacy-close']//i[@class='icon-close']")
    close_privacy.click()
    
    time.sleep(0.5)
    for year in list_years:
        masters_year, driver = pull_one_year(year,driver)
        masters_results = pd.concat([masters_results,masters_year])
    
    driver.quit()
    return(masters_results)

In [40]:
masters_data = masters_all_years(years,df_headers)

<ipython-input-37-2028e45b613d>:9: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  close_privacy = driver.find_element_by_xpath("//div[@class= 'privacy-close']//i[@class='icon-close']")
<ipython-input-38-307c802c3cc5>:11: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  down_button = driver.find_element_by_xpath("//div[@class = 'title navigation_down_arrow']")
<ipython-input-38-307c802c3cc5>:16: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  year_button = driver.find_element_by_xpath(f"//div[@class='year-list list1']//li[text() = {input_year}]")


In [41]:
masters_winners = masters_data.loc[masters_data["Pos"] == '1']

In [43]:
#masters_data.to_csv("masters_data.csv")
#masters_winners.to_csv("masters_winners.csv")